In [2]:
from transformers import pipeline
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain
import os
import json
import re
import docx

def env_vars():
    with open('.env', 'r') as f:
        lines = f.readlines()
    
    lines = [re.sub('"|\n', '', x) for x in lines]
    lines = [{x.split('=')[0]: x.split('=')[1]} for x in lines]

    env_object = {}
    for json_ in lines:
        for key, value in json_.items():
            env_object[key] = value

    return env_object

ENV_VARS = env_vars()
OPEN_AI_KEY=ENV_VARS['OPEN_AI_KEY']

os.environ['OPENAI_API_KEY'] = OPEN_AI_KEY

In [40]:
def chunk_string(s, chunk_size):
    # Split the string into words
    words = s.split()

    # Create chunks of words
    chunks = [words[i:i + chunk_size] for i in range(0, len(words), chunk_size)]

    # Join the chunks back into strings
    return [' '.join(chunk) for chunk in chunks]

def create_table_of_contents(sampled_text):
    create_topics_template = """
        Question: Please Create a table of contents, with section titles summarizing chunks of text. Each section will start with a number (1-5) (at most 5 sections), and each section description should be less than 50 characters, for this text: {text}.

        Please return the response as a json object, where the key is the first 40 chars of the sentence where the section starts, and the value is the section title. 
        
        Answer: Here is the response json object:
    """
    openai_llm = OpenAI(verbose=True, temperature=.1, model_name="text-davinci-003")
    simple_prompt = PromptTemplate(input_variables=["text"], template=create_topics_template)
    chain = LLMChain(llm=openai_llm, prompt=simple_prompt)
    table_of_contents = chain.run(sampled_text)
    table_of_contents = json.loads(table_of_contents)

    print(table_of_contents)

    #
    new_table_of_contents = {}

    counter = 0
    for x in table_of_contents.items():
        if counter == 0:
            new_table_of_contents[x[0]] = x[1] + '\n\n' + x[0]

        else:
            new_table_of_contents[x[0]] = '\n\n' + x[1] + '\n\n' + x[0]

        counter += 1
    
    return new_table_of_contents

#write a function to write text to a word docx file
def write_to_word(text, filename):
    doc = docx.Document()
    doc.add_paragraph(text)
    doc.save(filename)

In [10]:
file = "input_data/Florida Gov. Ron DeSantis Vows 'No More Socialism For The Wealthy' During Economic Speech.mp4"
# file = "FULL SPEECH _ Presidential candidate Ron Desantis at NC GOP Convention.mp4"

pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-large-v2",
  chunk_length_s=30,
  device="cpu",
)

prediction = pipe(file, batch_size=8)["text"]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [4]:
speech_chunks = chunk_string(prediction, 600)
toc_list = [create_table_of_contents(x) for x in speech_chunks[0:3]]
toc_list

NameError: name 'prediction' is not defined

{'Number nine Ensure that bad economic': 'Economic Policy\n\nNumber nine Ensure that bad economic',
 'Part of having a good economy is that': 'Cycles in the Economy\n\nPart of having a good economy is that',
 'Of course, the people that are not': 'Elites Bailed Out\n\nOf course, the people that are not',
 "We know that that's not the case.": "No Bailouts\n\nWe know that that's not the case.",
 'And finally, number 10, we need to': "Congress's Spending Habits\n\nAnd finally, number 10, we need to"}

In [42]:
speech_chunks = chunk_string(prediction, 600)
_test = create_table_of_contents(speech_chunks[0])
_test

test_chunk = speech_chunks[0]
test_chunk

for x in _test.items():
    test_chunk = re.sub(x[0], x[1], test_chunk)

test_chunk

write_to_word(test_chunk, 'test_chunk.docx')

{'Number nine Ensure that bad economic': 'Bad Economic Actors Held Responsible', 'Part of having a good economy is': 'Good Economy Has Cycles', 'When elites get bailed out from': 'Elites Bailed Out Distorts Economy', 'And finally, number 10, we need': "Rein In Congress's Spending Habits", 'So at the end of the day, economic': 'Economic Policy Beyond Dollars & Cents'}


In [41]:
_test = create_table_of_contents(speech_chunks[0])
_test

{'Number nine Ensure that bad economic': 'Bad Economic Actors Held Responsible', 'Part of having a good economy is that': 'Good Economy Has Cycles', 'When elites get bailed out from their': 'Elites Bailed Out from Bad Decisions', 'And finally, number 10, we need to': "Rein in Congress's Spending Habits", 'So at the end of the day, economic': 'Economic Policy Beyond Dollars & Cents'}


{'Number nine Ensure that bad economic': 'Bad Economic Actors Held Responsible\n\nNumber nine Ensure that bad economic',
 'Part of having a good economy is that': '\n\nGood Economy Has Cycles\n\nPart of having a good economy is that',
 'When elites get bailed out from their': '\n\nElites Bailed Out from Bad Decisions\n\nWhen elites get bailed out from their',
 'And finally, number 10, we need to': "\n\nRein in Congress's Spending Habits\n\nAnd finally, number 10, we need to",
 'So at the end of the day, economic': '\n\nEconomic Policy Beyond Dollars & Cents\n\nSo at the end of the day, economic'}

In [45]:
#write text to a pdf file

KeyError: 0